In [1]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Declare constants
training_source = 'Linux Academy'
tech_name = 'AWS'
no_of_courses = 6

# course id
aws_certified_cloud_practitioner = 0
aws_soultions_architect_associate = 1
aws_developer_associate = 2
aws_certified_sysops_administrator_associate = 3
aws_certified_devops_engineer_professional = 4
aws_certified_solutions_architect_professional=5

# Initial Arrays 
exam_names = ['']*no_of_courses
exam_codes = ['']*no_of_courses
urls =['']*no_of_courses

# Course names
exam_names[aws_certified_cloud_practitioner] ='aws-certified-cloud-practitioner'
exam_names[aws_developer_associate] ='aws-developer-associate'
exam_names[aws_soultions_architect_associate] ='aws-soultions-architect-associate' 
exam_names[aws_certified_sysops_administrator_associate]='aws-certified-sysops-administrator-associate'
exam_names[aws_certified_devops_engineer_professional]='aws-certified-devops-engineer-professional'
exam_names[aws_certified_solutions_architect_professional]='aws-certified-solutions-architect-professional'

# Exam Codes
exam_codes[aws_certified_cloud_practitioner] ='CLF-C01'
exam_codes[aws_developer_associate] ='DVA-C01'
exam_codes[aws_soultions_architect_associate] ='SAA-C02'  
exam_codes[aws_certified_sysops_administrator_associate]='SOA-C01'
exam_codes[aws_certified_devops_engineer_professional]='DOP-C01'
exam_codes[aws_certified_solutions_architect_professional]='SAP-C01'

# Course urls
urls[aws_certified_cloud_practitioner]='https://linuxacademy.com/course/aws-cloud-practitioner-clf-c-01/'

urls[aws_developer_associate] = 'https://linuxacademy.com/course/aws-certified-developer-associate-2018/'
urls[aws_soultions_architect_associate] ='https://linuxacademy.com/course/aws-certified-solutions-architect-associate-level/'
urls[aws_certified_sysops_administrator_associate]='https://linuxacademy.com/course/aws-certified-sys-ops-administrator-associate-soa-c-01/'

urls[aws_certified_devops_engineer_professional]='https://linuxacademy.com/course/aws-certified-dev-ops-professional-exam-prep-course/'
urls[aws_certified_solutions_architect_professional]='https://linuxacademy.com/course/aws-certified-solutions-architect-professional/'



In [3]:
# Set Current Course & URL 
current_exam_scraped = aws_certified_devops_engineer_professional
exam_code = exam_codes[current_exam_scraped]
exam_name = exam_names[current_exam_scraped]
start_url = urls[current_exam_scraped]

# Set the file name
file_name = training_source+'-'+exam_name

In [4]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(10)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)
          

In [5]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read()  

In [6]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('TECH')
table_columns.append('EXAM CODE')
table_columns.append('TRAINING PORTAL')
table_columns.append('EXAM NAME')
table_columns.append('COURSE NAME')
table_columns.append('TITLE ORDER')
table_columns.append('TITLE')
table_columns.append('NO OF SUB-SECTIONS')
table_columns.append('SUB TITLE')
table_columns.append('DURATION')

table_data = []
title_order = 0

# Grap the course name 
course_name = soup.select('h1[class="f6 f4-l flex-auto"]')[0].text


# Select course list
course_content = soup.select('h5[class="f11 f10-l mt4 mb2"]') 
course_sub_sections = soup.select('div[class*="course-syllabus mt4 mt7-l"]')

for index,title in enumerate(course_content):
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    title = title.text
    duration = ''
    
    # Set the row data
    row_data.append(tech_name)
    row_data.append(exam_code)
    row_data.append(training_source)
    row_data.append(exam_name)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(duration)
    table_data.append(row_data)
   
    sub_sections_count = 0

    topic_details = course_sub_sections[0].select('div[class*="course-syllabus-section mb4"]')
    sub_section_titles = topic_details[index].select('div[class*="course-title"]')
    sub_section_durations = topic_details[index].select('div[class*="course-duration"]')
    
    # Loop to get sub section details 
    for sub_index,sub_item in enumerate(sub_section_titles):
        sub_sections_count +=1
        sub_row = row_data.copy()
        
        # Extract data 
        sub_title = sub_item.select('p')[0].text.strip()
        sub_duration = sub_section_durations[sub_index].select('p')[0].text.strip()
        
        sub_row[8] = sub_title
        sub_row[9] = sub_duration
        table_data.append(sub_row)
    row_data[7] =sub_sections_count
 

In [7]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,TECH,EXAM CODE,TRAINING PORTAL,EXAM NAME,COURSE NAME,TITLE ORDER,TITLE,NO OF SUB-SECTIONS,SUB TITLE,DURATION
0,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,13,,
1,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,DevOps Fundamentals Scenarios,00:07:45
2,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,SDLC Automation Scenarios,00:05:18
3,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,Infrastructure As Code And Related Scenarios,00:06:58
4,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,OpsWorks Scenarios,00:06:35
5,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,Auto Scaling Scenarios,00:07:07
6,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,EC2 Deployment Scenarios,00:05:09
7,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,Lambda and API Gateway Scenarios,00:05:09
8,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,Blue/Green and A/B Testing Scenarios,00:08:14
9,AWS,DOP-C01,Linux Academy,aws-certified-devops-engineer-professional,AWS Certified DevOps Professional Exam Prep Co...,1,Whiteboarding Real World Scenarios,,Monitoring and Logging Scenarios,00:04:13
